## Знакомство с данными

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [2]:
# выгружаем наши данные
query_1 = 'select * from books'
books = pd.io.sql.read_sql(query_1, con = engine)

query_2 = 'select * from authors'
authors = pd.io.sql.read_sql(query_2, con = engine)

query_3 = 'select * from publishers'
publishers = pd.io.sql.read_sql(query_3, con = engine)

query_4 = 'select * from ratings'
ratings = pd.io.sql.read_sql(query_4, con = engine)

query_5 = 'select * from reviews'
reviews = pd.io.sql.read_sql(query_5, con = engine)

In [3]:
list = [books, authors, publishers, ratings, reviews]

for data in list:
    print(data.info())
    print('Количество дубликатов в данных: {}'.format(data.duplicated().sum()))
    print()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
book_id             1000 non-null int64
author_id           1000 non-null int64
title               1000 non-null object
num_pages           1000 non-null int64
publication_date    1000 non-null object
publisher_id        1000 non-null int64
dtypes: int64(4), object(2)
memory usage: 47.0+ KB
None
Количество дубликатов в данных: 0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
author_id    636 non-null int64
author       636 non-null object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB
None
Количество дубликатов в данных: 0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
publisher_id    340 non-null int64
publisher       340 non-null object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB
None
Количество дубликатов в данных: 0

<class 'pandas.core.frame.DataFrame'>

***Описание данных***

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

Пропущенных значений нет, дубликатов также не обнаружили.
Тип данны соответствует смысловой нагрузке каждого столбца, кроме столбца с датой. При решении поставленных вопросов будем предобразовывать типа данных в столбцах с датой и можем приступать к анализу.

## Количество книг, выпущенных после 1 января 2000 года

In [4]:
def select(sql):
  return pd.io.sql.read_sql(sql, con = engine)

In [5]:
sql = '''SELECT COUNT(*) AS cnt
    FROM books
    WHERE cast(publication_date as date) > '2000-01-01'
'''
select(sql)

,cnt
0,819


С начала 2000 года было выпущено 819 книг.

## Количество обзоров и средняя оценка для каждой книги

In [6]:
sql = '''SELECT COUNT(DISTINCT reviews) AS review_id_cnt
    FROM reviews
'''
select(sql)

,review_id_cnt
0,2793


In [7]:
sql = '''SELECT 
            *
        FROM 
            books
        INNER JOIN reviews ON reviews.book_id = books.book_id
        INNER JOIN ratings ON ratings.book_id = books.book_id
        LIMIT 5;
'''
select(sql)

,book_id,author_id,title,num_pages,publication_date,publisher_id,review_id,book_id,username,text,rating_id,book_id,username,rating
0,1,546,'Salem's Lot,594,2005-11-01,93,1,1,brandtandrea,Mention society tell send professor analysis. ...,1,1,ryanfranco,4
1,1,546,'Salem's Lot,594,2005-11-01,93,1,1,brandtandrea,Mention society tell send professor analysis. ...,2,1,grantpatricia,2
2,1,546,'Salem's Lot,594,2005-11-01,93,1,1,brandtandrea,Mention society tell send professor analysis. ...,3,1,brandtandrea,5
3,1,546,'Salem's Lot,594,2005-11-01,93,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...,1,1,ryanfranco,4
4,1,546,'Salem's Lot,594,2005-11-01,93,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...,2,1,grantpatricia,2


In [296]:
sql = '''SELECT 
            books.book_id AS book_id,
            COUNT(DISTINCT reviews.review_id) AS review_id_cnt,
            AVG(ratings.rating) AS rating_avg
        FROM 
            books
        INNER JOIN reviews ON reviews.book_id = books.book_id
        INNER JOIN ratings ON ratings.book_id = books.book_id
        GROUP BY 
            books.book_id
        LIMIT 5;
'''
select(sql)

,book_id,review_id_cnt,rating_avg
0,1,2,3.666667
1,2,1,2.500000
2,3,3,4.666667
3,4,2,4.500000
4,5,4,4.000000


In [307]:
sql = '''SELECT 
            books.book_id AS book_id,
            COUNT(DISTINCT reviews.review_id) AS review_id_cnt,
            AVG(ratings.rating) AS rating_avg
        FROM 
            books
        INNER JOIN reviews ON reviews.book_id = books.book_id
        INNER JOIN ratings ON ratings.book_id = books.book_id
        GROUP BY 
            books.book_id
;
'''
select(sql).review_id_cnt.sum()

2793

In [308]:
# КОД РЕВЬЮЕРА

sql = '''SELECT 
            books.book_id AS book_id,
            COUNT(reviews.review_id) AS review_id_cnt,
            AVG(ratings.rating) AS rating_avg
        FROM 
            books
        INNER JOIN reviews ON reviews.book_id = books.book_id
        INNER JOIN ratings ON ratings.book_id = books.book_id
        GROUP BY 
            books.book_id
;
'''
select(sql).review_id_cnt.sum()

26167

## Издательство, которое выпустило наибольшее число книг толще 50 страниц (исключаем брошюры)

In [10]:
sql = '''SELECT 
            publishers.publisher AS publisher,
            COUNT(books.title) AS cnt
        FROM 
            books
        INNER JOIN publishers ON publishers.publisher_id = books.publisher_id
        WHERE books.num_pages > 50
        GROUP BY 
            publishers.publisher
        ORDER BY 
            cnt DESC
        LIMIT 3;

'''
select(sql)

,publisher,cnt
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25


## Автор с самой высокой средней оценкой книг  
***учитываем только книги с 50 и более оценками***

In [299]:
sql = '''SELECT
            SBQ1.author_id AS author_id,
            AVG(SBQ1.rating_avg) AS rating_avg
        FROM
        
                    (SELECT 
                        books.book_id AS book_id,
                        authors.author AS author_id,
                        AVG(ratings.rating) AS rating_AVG           
                    FROM
                        books
                    INNER JOIN ratings ON ratings.book_id = books.book_id
                    INNER JOIN authors ON authors.author_id = books.author_id

                    WHERE books.book_id in
                                        (SELECT
                                            SBQ.book_id
                                        FROM         
                                            (SELECT
                                                books.book_id AS book_id,
                                                COUNT(rating_id) AS rating_cnt
                                            FROM 
                                                books
                                            INNER JOIN ratings ON ratings.book_id = books.book_id
                                            GROUP BY
                                                books.book_id) AS SBQ
                                  WHERE
                                     SBQ.rating_cnt >= 50)
                     GROUP BY
                            books.book_id,
                            authors.author_id
                     ORDER BY 
                        rating_avg DESC) AS SBQ1
            
        GROUP BY
            SBQ1.author_id
        
'''        
         
select(sql)  

,author_id,rating_avg
0,J.K. Rowling/Mary GrandPré,4.283844
1,Dan Brown,3.754540
2,J.D. Salinger,3.825581
3,William Golding,3.901408
4,Louisa May Alcott,4.192308
5,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
6,Stephenie Meyer,3.662500
7,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
8,Rick Riordan,4.080645
9,George Orwell/Boris Grabnar/Peter Škerl,3.729730


Самая высокая средняя оценка книги у J.K. Rowling/Mary GrandPré

## Среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [306]:
 sql = '''      
                SELECT
                   reviews.username,
                   COUNT(reviews.text) AS text_cnt
                FROM
                    reviews
                WHERE reviews.username in
                            (SELECT
                                SBQ.username
                             FROM         
                                        (SELECT
                                            username,
                                            COUNT(rating_id) AS rating_cnt
                                        FROM 
                                            ratings
                                        GROUP BY
                                            username) AS SBQ
                              WHERE
                                 SBQ.rating_cnt > 50)
                GROUP BY
                     reviews.username
                     
'''        
         
select(sql)

,username,text_cnt
0,sfitzgerald,28
1,jennifermiller,25
2,xdavis,18
3,paul88,22
4,martinadam,27
5,richard89,26


In [305]:
sql = '''SELECT
            AVG(SBQ1.text_cnt) AS text_cnt
        FROM
                (SELECT
                   reviews.username,
                   COUNT(reviews.text) AS text_cnt
                FROM
                    reviews
                WHERE reviews.username in
                            (SELECT
                                SBQ.username
                             FROM         
                                        (SELECT
                                            username,
                                            COUNT(rating_id) AS rating_cnt
                                        FROM 
                                            ratings
                                        GROUP BY
                                            username) AS SBQ
                              WHERE
                                 SBQ.rating_cnt > 50)
                GROUP BY
                     reviews.username) AS SBQ1

'''        
         
select(sql)

,text_cnt
0,24.333333
